In [4]:
cd ../chapter5/models-master/tutorials/rnn/ptb

/home/wjj/TFbook/chapter5/models-master/tutorials/rnn/ptb


In [5]:
import time
import numpy as np
import tensorflow as tf
import reader

In [6]:
class PTBInput(object):
    
    def __init__(self,config,data,name=None):
        self.batch_size = batch_size = config.batch_size    #从config中读取参数存到本地变量
        self.num_steps = num_steps = config.num_steps
        self.epoch_size = (len(data) // batch_size - 1) // num_steps
        self.input_data, self.targets = reader.ptb_producer(data,batch_size,num_steps,name=name)

In [3]:
class PTBModel(object):
    def __init__(self,is_training,config,input_):
        self._input = input_
        
        batch_size = input_.batch_size    #从input_中读取参数存到本地变量
        num_steps = input_.num_steps
        size = config.hidden_size    #从config中读取参数存到本地变量，隐含节点个数
        vocab_size = config.vocab_size
    def lstm_cell():    #使用tf.contrib.rnn.BasicLSTMCell函数设置默认的LSTM单元
        return tf.contrib.rnn.BasicLSTMCell(size,forget_bias=0.0,state_is_tuple=True)    #size是隐含节点个数，forgets_bias是forget gate的bias，
    attn_cell = lstm_cell
    if is_training and config.keepprob < 1:    #如果是在训练状态且keepprob<1则在前面的lstm_cell之后接一个Dropout层
        def attn_cell():    #使用tf.contrib.rnn.DropoutWrapper函数设置一个dropout层
            return tf.contrib.rnn.DropoutWrapper(lstm_cell(),output_keep_prob=config.keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([attn_cell() for _ in range(config.num_layers)],state_is_tuple=True)#用tf.contrib.rnn.MultiRNNCell函数堆叠前面构造的lstm_cell
    
    self._initial_state = cell.zero_state(batch_size, tf.float32)    #设置LSTM单元的初始化状态为0
    
    with tf.device("/")